In [4]:
# ==============================================================================
# CÉLULA 1: SETUP E CONFIGURAÇÃO
# ==============================================================================
import sys
import time
import gc
import yaml
import numpy as np
import pandas as pd
from pathlib import Path
import os

# Imports matemáticos vitais
from scipy.sparse import load_npz, save_npz, csr_matrix, vstack
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns

def find_project_root(anchor_file="conf/config.yaml"):
    current_path = Path.cwd()
    for parent in [current_path] + list(current_path.parents):
        if (parent / anchor_file).exists(): return parent
    raise FileNotFoundError(f"Raiz do projeto não encontrada.")

# 1. Carregar Configuração
try:
    BASE_DIR = find_project_root()
    print(f"📂 Raiz: {BASE_DIR}")
except:
    BASE_DIR = Path("/Users/lucasborges/Downloads/TCC") # Fallback
    print(f"⚠️ Fallback: {BASE_DIR}")

with open(BASE_DIR / "conf/config.yaml", "r") as f:
    config = yaml.safe_load(f)

P = {k: BASE_DIR / v for k, v in config['paths'].items()}

# 2. Mapear Arquivos (Define a variável PATHS)
print("Mapeando arquivos...")
PATHS = {
    "B_lcc":       P['graphs_bipartite'] / "B_lcc.npz",
    "m_index":     P['graphs_bipartite'] / "m_index_lcc.parquet",
    "B_coarse":    P['graphs_coarsened'] / "B_coarsened.npz",
    "s_index":     P['graphs_coarsened'] / "super_m_index.parquet"
}

# 3. Mapear Saídas
OUT_ORIG_DIR  = P.get('graphs_item', BASE_DIR / "graphs/item_item")
OUT_SUPER_DIR = P.get('graphs_super', BASE_DIR / "graphs/super_item_item")

OUT_ORIG_DIR.mkdir(parents=True, exist_ok=True)
OUT_SUPER_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Setup concluído! Variável PATHS definida.")

📂 Raiz: /Users/lucasborges/Downloads/TCC
Mapeando arquivos...
✅ Setup concluído! Variável PATHS definida.


In [5]:
# --- FUNÇÃO DE PROJEÇÃO ---
def project_and_sparsify(
    B,
    k_neighbors=100,
    chunk_size=1000, # Processar em lotes para não estourar RAM
    verbose_prefix="",
):
    """
    1. Projeta B (P x M) -> S (M x M) via Cosine Similarity.
    2. Mantém apenas os Top-K vizinhos por nó (KNN).
    """
    n_rows, n_cols = B.shape # n_cols são os itens (músicas)
    
    print("\n" + "="*70)
    print(f"{verbose_prefix} PROJEÇÃO (COSINE) + SPARSIFICATION (KNN={k_neighbors})")
    print("="*70)
    print(f"Input B: {n_rows:,} playlists × {n_cols:,} itens")
    
    # 1. Normalização L2 nas Colunas (Itens)
    # Transpomos para X (M x P) para normalizar as linhas (cada linha = 1 música)
    # Cosine Similarity(A, B) = (A . B) / (||A||*||B||)
    # Se normalizarmos vetores para tamanho 1, Cosine = A . B
    print("Normalizando vetores de itens (L2)...")
    X = B.T.tocsr() # Shape: (Itens, Playlists)
    X = normalize(X, norm='l2', axis=1) # Cada música agora é um vetor unitário
    
    # 2. Cálculo de Similaridade e Filtragem Top-K em Blocos
    # Não podemos fazer X.dot(X.T) direto se X for gigante (resultado denso).
    # Vamos calcular bloco a bloco.
    
    print(f"Calculando similaridade e filtrando Top-{k_neighbors} em blocos de {chunk_size}...")
    
    sparse_blocks = []
    start_time = time.time()
    
    # Iterar sobre os itens em pedaços (chunks)
    for i in range(0, n_cols, chunk_size):
        end = min(i + chunk_size, n_cols)
        
        # A. Pega um lote de músicas
        X_chunk = X[i:end]
        
        # B. Calcula similaridade deste lote contra TODAS as músicas
        # Resultado: (chunk_size, n_cols)
        # Valores entre 0 e 1
        sim_chunk = X_chunk.dot(X.T)
        
        # C. Zerar diagonal (apenas para o bloco onde i==j)
        # O offset da diagonal neste bloco depende de 'i'
        sim_chunk.setdiag(0) # Scipy sparse matrix cuida do offset automaticamente se for quadrada, mas aqui é retangular
        # Correção manual para diagonal:
        # Se o bloco contém a diagonal principal da matriz global
        if i < end: 
             # Zera auto-loops na força bruta (seguro e rápido para chunks)
             # Criamos uma máscara para (r, c) onde r+i == c
             for row_idx in range(sim_chunk.shape[0]):
                 global_idx = i + row_idx
                 sim_chunk[row_idx, global_idx] = 0.0

        # D. Manter apenas Top-K para cada linha do chunk
        # Converter para denso temporariamente é seguro pois chunk_size é pequeno
        sim_dense = sim_chunk.toarray() 
        
        # Lógica de seleção Top-K usando argpartition (rápido)
        # Cria uma matriz de zeros
        filtered_data = np.zeros_like(sim_dense)
        
        for row_idx in range(len(sim_dense)):
            row = sim_dense[row_idx]
            # Se tem mais vizinhos que K, filtra
            if np.count_nonzero(row) > k_neighbors:
                # Índices dos K maiores
                top_k_idx = np.argpartition(row, -k_neighbors)[-k_neighbors:]
                filtered_data[row_idx, top_k_idx] = row[top_k_idx]
            else:
                filtered_data[row_idx] = row
        
        # Converter de volta para esparso e guardar
        sparse_blocks.append(csr_matrix(filtered_data))
        
        if (i // chunk_size) % 10 == 0:
            print(f"   Processado {end}/{n_cols} itens... ({time.time()-start_time:.1f}s)")
            gc.collect()

    print("Empilhando blocos finais...")
    S = vstack(sparse_blocks)
    S.eliminate_zeros()
    
    # Estatísticas Finais
    n = S.shape[0]
    deg = np.diff(S.indptr)
    print(f"✓ Concluído: {n:,} × {n:,}")
    print(f"  NNZ Total: {S.nnz:,}")
    print(f"  Grau Médio: {deg.mean():.2f}")
    print(f"  Grau Max: {deg.max()} (Deve ser <= {k_neighbors})")
    
    return S

In [6]:
print("\nCarregando matriz B_lcc (playlists × músicas)...")

# USANDO A NOVA CONFIGURAÇÃO DINÂMICA
B_lcc = load_npz(PATHS["B_lcc"])

m_index_orig = pd.read_parquet(PATHS["m_index"]).squeeze()

# Ajuste de robustez para index
if isinstance(m_index_orig, pd.DataFrame): m_index_orig = m_index_orig.iloc[:, 0]
if isinstance(m_index_orig, pd.Series): m_index_orig = pd.Index(m_index_orig)

# --- PROJEÇÃO ---
# K=100 é um bom balanço para músicas. 
# O chunk_size=1000 garante que não trave sua memória.
S_tracks = project_and_sparsify(
    B=B_lcc,
    k_neighbors=100, 
    chunk_size=1000, 
    verbose_prefix="MÚSICA–MÚSICA (KNN)"
)

# --- SALVAMENTO ---
# Usamos OUT_ORIG_DIR (definido na Célula 1)
save_npz(OUT_ORIG_DIR / "A_tracks_adjacency.npz", S_tracks)
m_index_orig.to_frame(name="track_uri").to_parquet(OUT_ORIG_DIR / "m_index_lcc_tracks.parquet")

print(f"\n✓ Projeção música–música salva em: {OUT_ORIG_DIR}")


Carregando matriz B_lcc (playlists × músicas)...

MÚSICA–MÚSICA (KNN) PROJEÇÃO (COSINE) + SPARSIFICATION (KNN=100)
Input B: 98,726 playlists × 324,132 itens
Normalizando vetores de itens (L2)...
Calculando similaridade e filtrando Top-100 em blocos de 1000...
   Processado 1000/324132 itens... (6.5s)
   Processado 11000/324132 itens... (64.0s)
   Processado 21000/324132 itens... (119.7s)
   Processado 31000/324132 itens... (175.0s)
   Processado 41000/324132 itens... (229.3s)
   Processado 51000/324132 itens... (283.6s)
   Processado 61000/324132 itens... (339.1s)
   Processado 71000/324132 itens... (395.1s)
   Processado 81000/324132 itens... (449.4s)
   Processado 91000/324132 itens... (503.1s)
   Processado 101000/324132 itens... (558.8s)
   Processado 111000/324132 itens... (613.7s)
   Processado 121000/324132 itens... (669.5s)
   Processado 131000/324132 itens... (722.3s)
   Processado 141000/324132 itens... (774.4s)
   Processado 151000/324132 itens... (826.7s)
   Processado 161

In [7]:
print("\nCarregando matriz B_coarsened (playlists × super-músicas)...")

# USANDO A NOVA CONFIGURAÇÃO DINÂMICA
B_coarsened = load_npz(PATHS["B_coarse"])

super_m_index = pd.read_parquet(PATHS["s_index"]).squeeze()

if isinstance(super_m_index, pd.DataFrame): super_m_index = super_m_index.iloc[:, 0]
if isinstance(super_m_index, pd.Series): super_m_index = pd.Index(super_m_index)

# --- PROJEÇÃO ---
# K=50 para super-nós (grafo menor, estrutura mais rígida)
S_super = project_and_sparsify(
    B=B_coarsened,
    k_neighbors=50,
    chunk_size=2000, 
    verbose_prefix="SUPER-MÚSICA (KNN)"
)

# --- SALVAMENTO ---
# Usamos OUT_SUPER_DIR (definido na Célula 1)
save_npz(OUT_SUPER_DIR / "A_super_tracks_adjacency.npz", S_super)
super_m_index.to_frame(name="super_track_id").to_parquet(OUT_SUPER_DIR / "super_m_index.parquet")

print(f"\n✓ Projeção super-música salva em: {OUT_SUPER_DIR}")


Carregando matriz B_coarsened (playlists × super-músicas)...

SUPER-MÚSICA (KNN) PROJEÇÃO (COSINE) + SPARSIFICATION (KNN=50)
Input B: 98,726 playlists × 20,047 itens
Normalizando vetores de itens (L2)...
Calculando similaridade e filtrando Top-50 em blocos de 2000...
   Processado 2000/20047 itens... (0.7s)
   Processado 20047/20047 itens... (8.6s)
Empilhando blocos finais...
✓ Concluído: 20,047 × 20,047
  NNZ Total: 1,002,350
  Grau Médio: 50.00
  Grau Max: 50 (Deve ser <= 50)

✓ Projeção super-música salva em: /Users/lucasborges/Downloads/TCC/graphs/super_item_item


In [8]:
def summarize_matrix(name, S):
    n = S.shape[0]
    nnz = S.nnz
    density = nnz / (n * n)
    deg = np.asarray(S.sum(axis=1)).ravel()
    print(f"\n--- {name} ---")
    print(f"Shape: {n:,} × {n:,}")
    print(f"nnz: {nnz:,}")
    print(f"Densidade: {density:.8e}")
    print(f"Grau médio: {deg.mean():.2f}")
    print(f"Grau mediano: {np.median(deg):.2f}")
    print(f"Grau [min, max]: [{deg.min():.2f}, {deg.max():.2f}]")

summarize_matrix("Música–música (original)", S_tracks)
summarize_matrix("Super-música–super-música (coarsenado)", S_super)


--- Música–música (original) ---
Shape: 324,132 × 324,132
nnz: 23,684,266
Densidade: 2.25432284e-04
Grau médio: 28.62
Grau mediano: 21.65
Grau [min, max]: [0.02, 100.00]

--- Super-música–super-música (coarsenado) ---
Shape: 20,047 × 20,047
nnz: 1,002,350
Densidade: 2.49413877e-03
Grau médio: 4.96
Grau mediano: 4.38
Grau [min, max]: [1.27, 23.61]
